<a href="https://colab.research.google.com/github/Artur-Wanderley/Portfolio-of-my-data-science-projects/blob/master/Introdu%C3%A7%C3%A3_an%C3%A1lise_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ESPECIALIZAÇÃO EM ANÁLISE E ENGENHARIA DE DADOS - Cesar School
## Aluno: Artur Maia Wanderley
# Disciplina: Introdução à Ciência dos dados
# Projeto: Explorando os dados de acidentes de trânsito no município de Recife em 2018

## Introdução
O objetivo deste projeto é explorar os dados referentes aos acidentes de trânsito com e sem vítimas no município de Recife no ano de 2018. 

Este projeto está dividido em três partes:

- Obtenção dos dados
- Pré-processamento dos dados
  - Definição de tipos
  - Tratamento de dados ausentes
  - Normalização e discretização
  - Limpeza de dados (univariado e bivariado)
- Estatística descritiva e visualização dos dados

In [0]:
# Importando as bibliotecas que serão utilizadas nesse projeto
import pandas as pd
import numpy as np

# Obtenção dos dados
Os dados utilizados neste projeto foram obtidos no [Portal de Dados Abertos da Cidade do Recife](http://dados.recife.pe.gov.br/) e podem ser acessados [aqui](http://dados.recife.pe.gov.br/dataset/acidentes-de-transito-com-e-sem-vitimas/resource/2485590a-3b35-4ad0-b955-8dfc36b61021).

In [5]:
# Importando os dados
acidentes_rec=pd.read_csv("https://raw.githubusercontent.com/Artur-Wanderley/introducao-analise-de-dados/master/acidentes2018.csv",sep=";")
acidentes_rec.head()

,tipo,situacao,data,hora,bairro,endereco,numero,complemento,natureza,descricao,auto,moto,ciclom,ciclista,pedestre,onibus,caminhao,viatura,outros,vitimas,vitimasfatais
0,ATROPELAMENTO,FINALIZADA,2018-01-01,0:01:00,SÃO JOSÉ,AV ENGENHEIRO JOSE ESTELITA,NaN,EM FRENTE AO CAIS JOSE ESTELITA,ATROPELAMENTO,AUTO E PEDESTRE C/V,1,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,1,NaN
1,COLISÃO,CANCELADA,2018-01-01,2:45:00,CORDEIRO,AV CAXANGA,NaN,"PROX DO GALETOS, EM FRENTE AO PARQUE DE EXPOSI...",COLISÃO,COLISÃO ENTRE AUTO E MO S/V,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
2,COLISÃO,CANCELADA,2018-01-01,3:33:00,BOA VIAGEM,AV CONSELHEIRO AGUIAR,NaN,"PROX DO BANCO DO BRASIL, PROX DO BANCO ITAÚ",COLISÃO,COLISÃO ENTRE UM AUTO(TAXI) E VIATURA DA AERON...,1,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,0,NaN
3,COLISÃO,FINALIZADA,2018-01-01,7:13:00,BAIRRO DO RECIFE,AV ALFREDO LISBOA,NaN,PROX AO POSTO DE GASOLINA BR AO LADO DO ...,COLISÃO,COLISÃO SEM VITIMAS,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN
4,COLISÃO,FINALIZADA,2018-01-01,7:50:00,CASA AMARELA,RUA PADRE LEMOS,767,EM FRENTE AO POSTO PETROBRAS,COLISÃO,TAXI E AUTO,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN


# Definição dos tipos de dados

In [7]:
acidentes_rec.dtypes

tipo              object
situacao          object
data              object
hora              object
bairro            object
endereco          object
numero            object
complemento       object
natureza          object
descricao         object
auto              object
moto             float64
ciclom           float64
ciclista         float64
pedestre          object
onibus           float64
caminhao         float64
viatura           object
outros            object
vitimas           object
vitimasfatais    float64
dtype: object

Existem dois tipos de dados em `acidentes_rec`, `object`, `float64`. 

Como vemos acima, as colunas `tipo` e `situacao` podem ser transformadas em variáveis categóricas. Vamos olhar quais quais valores essas colunas trazem.

Já as colunas `data` e `hora` devem ser convertidas em tipo `datetime`.

As colunas `bairro`, `endereco`, `numero`, `complemento`, `natureza` e `descricao` podem ser mantidos como tipo `object` já que elas são strings que contêm informações acerca do local e das circunstâncias em que as colisões ocorreram. 

As demais colunas informam o número de veículos de diferentes tipos, bem como de pedestres e vítimas envolvidos nas colisões. Vamos transformar os valores de todas essas colunas em números inteiros (`int64`).

## Convertendo as colunas `tipo` e `situacao` em variáveis categóricas

In [16]:
# Observando os valores únicos na coluna tipo
unicos_tipo=acidentes_rec.tipo.value_counts()
print(unicos_tipo)
print("\nNúmero de valores únicos: ",unicos_tipo.shape[0])

COLISÃO                      8604
ABALROAMENTO LONGITUDINAL     704
COLISÃO TRASEIRA              524
ABALROAMENTO TRANSVERSAL      381
ATROPELAMENTO                 213
CHOQUE                        196
ENGAVETAMENTO                 172
COLISÃO COM CICLISTA          163
CHOQUE VEÍCULO PARADO          78
ACID. DE PERCURSO              60
COLISÃO FRONTAL                49
CHOQUE OBJETO FIXO             38
CAPOTAMENTO                    22
QUEDA DE ÁRVORE                21
ATROPELAMENTO ANIMAL            7
TOMBAMENTO                      7
0                               1
Name: tipo, dtype: int64

Número de valores únicos:  17


Como podemos ver, existem 17 valores únicos na coluna tipo que se referem a categorias distintas de colisão. Note que existe uma categoria cujo valor é `0`. Mais adiante investigaremos se esse valor é um erro que devemos remover do nosso dataset. 

Por enquanto, vamos transformar a coluna `tipo` em uma variável categórica com 17 categorias distintas de colisão. 

In [0]:
# Convertendo a coluna tipo em uma variável categóricas
acidentes_rec.tipo=acidentes_rec.tipo.astype("category")

Agora vamos ver os valores únicos da coluna `situcao`.

In [23]:
unicos_situacao=acidentes_rec.situacao.value_counts()
print(unicos_situacao)
print("\nNúmero de valores únicos: ",unicos_situacao.shape[0])

FINALIZADA         9537
CANCELADA          1693
PENDENTE             46
EQUIPE NO LOCAL       5
Name: situacao, dtype: int64

Número de valores únicos:  4


Como podemos ver, a coluna `situacao` possui valores únicos que indicam quatro  status de atendimento das ocorrências pelas autoridades.

Vamos transformar a coluna `situacao` em uma variável categórica.

In [0]:
# Convertendo a coluna situacao em variável categórica
acidentes_rec.situacao=acidentes_rec.situacao.astype("category")

## Convertendo as colunas `data` e `hora` em tipo `datetime`

In [0]:
acidentes_rec.data=pd.to_datetime(acidentes_rec.data,format="%Y-%m-%d")

As colunas `auto`,	`moto`,	`ciclom`,	`ciclista`,	`pedestre`,	`onibus`,	`caminhao`,	`viatura`,	`outros` possuem `NaN`, mas isso não significa que sejam missing data. Provavelmente, a maioria dos  `NaN` nessas colunas indicam que o tipo de veículo que intitula a coluna não esteve envolvido no acidente. Por exemplo, na primeira linha do dataset o acidente envolveu apenas um "auto" e um pedestre.

Dessa maneira, para essas colunas, vamos substituir os `NaN` por `0`. Esse procedimento tem o risco de substituir um missing data real por um valor irreal (`0`), mas se excluir as linhas com `NaN` perderemos um volume muito grande de dados.



In [0]:
# Subsituitindo os NaNs da coluna moto até a coluna outros

## Subsituitindo os NaNs da coluna `moto` até a coluna `outros`

In [0]:
acidentes_rec[["auto", "moto", "ciclom", "ciclista", "pedestre", "onibus", "caminhao", "viatura", "outros"]]=acidentes_rec[["auto", "moto", "ciclom", "ciclista", "pedestre", "onibus", "caminhao", "viatura", "outros"]].fillna(0)


In [0]:
acidentes_rec.head()

,tipo,situacao,data,hora,bairro,endereco,numero,complemento,natureza,descricao,auto,moto,ciclom,ciclista,pedestre,onibus,caminhao,viatura,outros,vitimas,vitimasfatais
0,ATROPELAMENTO,FINALIZADA,2018-01-01,0:01:00,SÃO JOSÉ,AV ENGENHEIRO JOSE ESTELITA,NaN,EM FRENTE AO CAIS JOSE ESTELITA,ATROPELAMENTO,AUTO E PEDESTRE C/V,1,0.0,0.0,0.0,1,0.0,0.0,0,0,1,NaN
1,COLISÃO,CANCELADA,2018-01-01,2:45:00,CORDEIRO,AV CAXANGA,NaN,"PROX DO GALETOS, EM FRENTE AO PARQUE DE EXPOSI...",COLISÃO,COLISÃO ENTRE AUTO E MO S/V,1,1.0,0.0,0.0,0,0.0,0.0,0,0,0,NaN
2,COLISÃO,CANCELADA,2018-01-01,3:33:00,BOA VIAGEM,AV CONSELHEIRO AGUIAR,NaN,"PROX DO BANCO DO BRASIL, PROX DO BANCO ITAÚ",COLISÃO,COLISÃO ENTRE UM AUTO(TAXI) E VIATURA DA AERON...,1,0.0,0.0,0.0,0,0.0,0.0,1,0,0,NaN
3,COLISÃO,FINALIZADA,2018-01-01,7:13:00,BAIRRO DO RECIFE,AV ALFREDO LISBOA,NaN,PROX AO POSTO DE GASOLINA BR AO LADO DO ...,COLISÃO,COLISÃO SEM VITIMAS,2,0.0,0.0,0.0,0,0.0,0.0,0,0,0,NaN
4,COLISÃO,FINALIZADA,2018-01-01,7:50:00,CASA AMARELA,RUA PADRE LEMOS,767,EM FRENTE AO POSTO PETROBRAS,COLISÃO,TAXI E AUTO,2,0.0,0.0,0.0,0,0.0,0.0,0,0,0,NaN


In [0]:
# Procurando missing data
acidentes_rec.isnull().sum()

tipo                42
situacao             1
data                 0
hora                13
bairro              26
endereco            18
numero            6037
complemento        437
natureza            42
descricao          227
auto              1021
moto              8648
ciclom           11228
ciclista         11122
pedestre         11078
onibus            9556
caminhao         10069
viatura          11120
outros           11152
vitimas             12
vitimasfatais    11255
dtype: int64

In [0]:
acidentes_rec.vitimasfatais.unique()

array([nan,  1.,  2.])